In [42]:
from builtins import Exception
from bs4 import BeautifulSoup
from requests import get
import mysql.connector
import datetime

# Class Scraping Data CNBC

class dibawah merupakan proses pengambilan data dari website berita CNBC yang dimana pemanggilan semua fungsi dibawah tersebut nantinya akan dipanggil pada fungsi [scrapingDataBeritaCNBC]

In [43]:
class CNBC:
    #deklarasi variable global
    url = ""
    mydb = ""
    keyword = ""
    urlDetailData = ""
    date = ""
    
    #fungsi dari inisialisasi data saat memanggil class pertama kali
    def __init__(self, urlBerita, pencarian,searchDate):
        self.url = urlBerita
        self.keyword = pencarian
        try:
            if(searchDate == ""):
                self.date = datetime.datetime.now()
            else:
                self.date = datetime.datetime.strptime(searchDate, '%d-%m-%Y')
        except Exception as e:
            print(e)
        self.mydb = mysql.connector.connect(host="localhost",user="root", password="",database="db-berita-mca")  
    
    #fungsi menyimpan data ke database
    def simpanDatabase(self,data):
        mycursor = self.mydb.cursor()
        sql = "INSERT INTO tb_berita (media,judul_berita,label,penulis,release_date,url,content) VALUES (%s,%s,%s,%s,%s,%s,%s)"
        mycursor.execute(sql, data)
        self.mydb.commit()
        return
    
    #fungsi buat validasi data duplicate di database
    def validasiDataDatabase(self,dataBerita):
        mycursor = self.mydb.cursor()
        sql = "SELECT * FROM tb_berita WHERE url = %s"
        dataUrl = dataBerita
        url = (dataBerita,)
        mycursor.execute(sql, url)
        myresult = mycursor.fetchall()
        return myresult
    
    #fungsi buat ngambil jumlah page pada pencarian berita tersebut
    def getAllPageWebsite(self):
        if(self.keyword == ""):
            urlGet = (self.url+"/indeks?date="+self.date.strftime("%Y/%m/%d"))
        else:
            urlGet = (self.url+"/search?query="+self.keyword+"&date="+self.date.strftime("%Y/%m/%d"))
        try:
            soup = BeautifulSoup(get(urlGet).text, 'lxml')
            getAllPage= soup.find('div','paging text_center gtm_paging')
            page_link = getAllPage.select('a')
        except Exception as e:
            print(e)
        try:
            for page in page_link:
                if(page.text != ""):
                    lastPage = page.text
            return lastPage
        except Exception as e:
            print("Tidak Terdapat Data pada Pencarian anda tersebut")
            
    
    #ngambil semua url-detail pada scraping data
    def getUrlInPage(self,page):
        self.urlDetailData = []
        for page in range(page):
            try:
                if(self.keyword == ""):
                    urlGet = (self.url+"/indeks/{}".format(page+1)+"?date="+self.date.strftime("%Y/%m/%d"))
    #                 print(urlGet)
                else:
                    urlGet = (self.url+"/search?query="+self.keyword+"&p={}".format(page+1)+"&date="+self.date.strftime("%Y/%m/%d"))
            except Exception as e:
                print("format tanggal anda masukan salah")
                break
            indeksPage = get(urlGet)
            soup = BeautifulSoup(indeksPage.text, 'html.parser')
            contents = soup.find_all('article')
            for content in contents:
                try:
                    news_url = content.find('a', href=True).get('href')
                    self.urlDetailData.append(news_url)
                except Exception as e:
                    continue
        return self.urlDetailData
    
    
    #Fungsi utama buat scraping semua data tersebut
    def scrapingDataBeritaCNBC(self):
        #disini aku deklarasiin manual pagenya kalau mau semua page berita yang diambil mati sudah ni engine wkwk
        self.getUrlInPage(5)
        for dataBerita in self.urlDetailData:
            myresult = self.validasiDataDatabase(dataBerita)
            if(len(myresult)==0):
                try:
                    web_data = get(dataBerita)
                    soup = BeautifulSoup(web_data.text, 'html.parser')
                    header = soup.find('div', class_='jdl')
                    media = "CBNC Indonesia"
                    title = header.find('h1').text
                    author_class = header.find('div', class_='author').text.split(' ')
                    label = author_class[0]
                    author = ' '.join(author_class[2:])
                    release_date = header.find('div', class_='date').text
                    detail_text_class = soup.find('div', class_='detail_text')
                    texts = detail_text_class.find_all('p')
                    news_content = ' '.join([text.text for text in texts])

                    data = (media,title,label,author,release_date,dataBerita,news_content)
                    self.simpanDatabase(data)
                except Exception as e:
                    continue    
            else:
                continue

In [46]:
url = "https://www.cnbcindonesia.com"
keyword = ""
# format tanggal day-month-years 25-12-2021
tanggal = "12-10-2021"
scrapCNBC = CNBC(url,keyword,tanggal)
scrapCNBC.getUrlInPage(3)

['https://www.cnbcindonesia.com/news/20211012220341-4-283438/percepat-klaim-bpjs-kesehatan-dorong-rs-lengkapi-berkas',
 'https://www.cnbcindonesia.com/news/20211012203456-4-283426/hampir-50-gadis-gadis-di-china-ogah-menikah-kenapa',
 'https://www.cnbcindonesia.com/market/20211012204726-17-283427/wall-street-dibuka-berayun-ke-zona-merah',
 'https://www.cnbcindonesia.com/news/20211012202743-4-283425/dana-mega-proyek-kereta-cepat-bengkak-begini-rinciannya',
 'https://www.cnbcindonesia.com/news/20211012202229-4-283424/parahnya-india-krisis-energi-gara-gara-batu-bara',
 'https://www.cnbcindonesia.com/news/20211012201327-4-283423/proyek-kereta-cepat-bengkak-ada-terowongan-tak-tembus-bor',
 'https://www.cnbcindonesia.com/news/20211012200640-4-283422/krisis-batu-bara-china-india-hadapi-krisis-energi-terburuk',
 'https://www.cnbcindonesia.com/news/20211012195932-4-283421/ada-ancaman-lebih-ngeri-dari-covid-asia-bisa-terendam-laut',
 'https://www.cnbcindonesia.com/tech/20211012164204-37-283375/ga